In [3]:
import pandas as pd
from collections import defaultdict

# Function to extract relevant data (dstport, protocol) from the message
def extract_flow_log_data(message):
    fields = message.split()
    
    # Ensure the log has the default AWS flow log format (version 2 has 14 fields)
    if len(fields) != 14:
        return None, None  # Skip logs that don't match the default format
    
    try:
        # Check the version; if it's not version 2, reject the log entry
        version = int(fields[0])
        if version != 2:
            return None, None

        dstport = int(fields[6])  # Destination port is in the 7th position (index 6)
        protocol_num = int(fields[7])  # Protocol is in the 8th position (index 7)
    except ValueError:
        # If we encounter an invalid value, we return None to skip this row
        return None, None

    # Protocol lookup based on protocol number
    protocol_mapping = {
        1: "icmp",
        6: "tcp",
        17: "udp",
        2: "igmp",
        41: "ipv6-encapsulation",
        47: "gre",
        50: "esp",
        51: "ah",
        58: "icmpv6",
        89: "ospf",
        88: "eigrp",
        132: "sctp",
        3: "ggp",
        4: "ipv4",
        8: "egp",
        12: "pup",
        27: "rdp",
        46: "rsvp",
        94: "ipip",
        115: "l2tp"
    }

    # Assign protocol based on the mapping; default to "other" if not found
    protocol = protocol_mapping.get(protocol_num, "other")
    
    return dstport, protocol

# Function to parse the flow logs and calculate tag counts and port/protocol counts
def parse_flow_logs(log_data, lookup_table):
    tag_counts = defaultdict(int)
    port_protocol_counts = defaultdict(int)
    untagged_count = 0

    for index, row in log_data.iterrows():
        message = row['message']
        dstport, protocol = extract_flow_log_data(message)

        if dstport is not None and protocol is not None:
            # Count port/protocol combination
            port_protocol_counts[(dstport, protocol)] += 1

            # Check if there's a matching tag in the lookup table
            matching_tag = lookup_table.get((dstport, protocol), "Untagged")
            if matching_tag != "Untagged":
                tag_counts[matching_tag] += 1
            else:
                untagged_count += 1

    return tag_counts, port_protocol_counts, untagged_count

# Function to write the output to CSV files
def write_output(tag_counts, port_protocol_counts, untagged_count):
    tag_counts_df = pd.DataFrame(list(tag_counts.items()), columns=['Tag', 'Count'])
    tag_counts_df.loc[len(tag_counts_df)] = ['Untagged', untagged_count]  # Add the untagged count
    tag_counts_df.to_csv('/Users/lakshmipriyankapenchikala/Desktop/tag_counts.csv', index=False)

    port_protocol_counts_df = pd.DataFrame(
        [(k[0], k[1], v) for k, v in port_protocol_counts.items()],
        columns=['Port', 'Protocol', 'Count']
    )
    port_protocol_counts_df.to_csv('/Users/lakshmipriyankapenchikala/Desktop/port_protocol_counts.csv', index=False)

# Function to load the lookup table from CSV
def load_lookup_table(filepath):
    lookup_df = pd.read_csv(filepath)
    lookup_table = {(int(row['dstport']), row['protocol']): row['tag'] for _, row in lookup_df.iterrows()}
    return lookup_table

# Main function to run the program
def main():
    # Define file paths
    lookup_table_file = '/Users/lakshmipriyankapenchikala/Desktop/lookup_table.csv'
    flow_log_file = '/Users/lakshmipriyankapenchikala/Desktop/log-events-viewer-result.csv'  # Change this path to your flow log file

    # Load lookup table
    lookup_table = load_lookup_table(lookup_table_file)

    # Load flow log data
    log_data = pd.read_csv(flow_log_file)

    # Parse flow logs and calculate tag counts and port/protocol counts
    tag_counts, port_protocol_counts, untagged_count = parse_flow_logs(log_data, lookup_table)

    # Write the results to CSV files
    write_output(tag_counts, port_protocol_counts, untagged_count)

if __name__ == '__main__':
    main()
